In [110]:
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.style.use('fivethirtyeight')

from os import listdir
from os.path import isfile, join

import re

In [111]:
def removeExtension(file_name):
    idx = -1
    for i, c in enumerate(file_name):
        if c == '.':
            idx = i
            break
    if idx >= 0:
        return file_name[:idx]
    return file_name

In [112]:
SUBMISSIONS_DIR = './submissions/'
MATCH_RESULTS_DIR = './match_results/'

In [113]:
files = sorted([f for f in listdir(SUBMISSIONS_DIR) if isfile(join(SUBMISSIONS_DIR, f)) and f[-1] != 'c'])
names = sorted([removeExtension(f) for f in files])

teams = list(zip(files, names))

In [118]:
match_results_df = pd.DataFrame(columns=['Team', 'Opponent', 'Wins', 'Losses', 'Ties', 'Invalid'])

In [ ]:
def process_raw_outcome(outcome):
    return outcome.replace('\n', '')

for i in range(len(teams)):
    for j in range(len(teams)):
        if i == j:
            continue
        
        file_a, name_a = teams[i]
        file_b, name_b = teams[j]
        
        # Get match results for team a vs team b
        match_data_file_name = name_a + '_vs_' + name_b + '.dat'
        try:
            match_data_file = open(join(MATCH_RESULTS_DIR, match_data_file_name), 'r')
        except FileNotFoundError:
            continue
        outcomes = [process_raw_outcome(l) for l in match_data_file]
        match_data_file.close()
        
        # Get DataFrame props
        num_rows, num_cols = match_results_df.shape
        
        # Insert row if doesn't exist or reset row
        index = -1
        for k in range(num_rows):
            row = match_results_df.loc[k]
            if row.Team == name_a and row.Opponent == name_b:
                index = k
                break
        if index == -1:
            index = num_rows
        
        win, loss, tied, invalid = 0, 0, 0, 0
        for outcome in outcomes:
            if outcome == name_a:
                win += 1
            elif outcome == name_b:
                loss += 1
            elif outcome == 'tied':
                tied += 1
            elif outcome == 'invalid':
                invalid += 1
            else:
                raise Error('Invalid match outcome')
        match_results_df.loc[index] = [name_a, name_b, win, loss, tied, invalid]

In [125]:
match_results_df.head()

,Team,Opponent,Wins,Losses,Ties,Invalid
0,PacBot_Kevin_Cao,baselineTeam2_Rachel_Li,9,3,0,3
1,PacBot_Kevin_Cao,baselineTeam2_Raghav_Anand,7,8,0,0
2,PacBot_Kevin_Cao,cleverTeam_Dylan_Rowe,10,5,0,0
3,baselineTeam2_Rachel_Li,contest_ANISH_SAHA,0,0,0,15
4,baselineTeam2_Raghav_Anand,contest_ANISH_SAHA_1,0,0,0,15


In [124]:
match_results_df[match_results_df['Invalid'] > 10]

,Team,Opponent,Wins,Losses,Ties,Invalid
3,baselineTeam2_Rachel_Li,contest_ANISH_SAHA,0,0,0,15
4,baselineTeam2_Raghav_Anand,contest_ANISH_SAHA_1,0,0,0,15
6,contest_ANISH_SAHA,myTeam2_4foodNum_ALI_KHALEEL_AHMED,0,0,0,15
7,contest_ANISH_SAHA_1,myTeam2_5foodNum_ALI_KHALEEL_AHMED,0,0,0,15
10,myTeam2_5foodNum_ALI_KHALEEL_AHMED,myTeam2_SEAN_ADAM_MEDIN,0,0,0,15
13,myTeam2_SEAN_ADAM_MEDIN,myTeam3_ALI_KHALEEL_AHMED,0,0,0,15
16,myTeam3_ALI_KHALEEL_AHMED,myTeam3_ALI_KHALEEL_AHMED_3,0,0,0,15
17,myTeam3_ALI_KHALEEL_AHMED_1,myTeam3_ALI_KHALEEL_AHMED_4,0,0,0,15
18,myTeam3_ALI_KHALEEL_AHMED_2,myTeam3_ALI_KHALEEL_AHMED_5,1,1,1,12
20,myTeam3_ALI_KHALEEL_AHMED_4,myTeam_ALI_KHALEEL_AHMED,0,0,0,15
